In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
import random
import shutil
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_directory = '/content/drive/MyDrive/Datasets/carro pronto/train'
test_directory = '/content/drive/MyDrive/Datasets/carro pronto/test'
val_directory = '/content/drive/MyDrive/Datasets/carro pronto/val'

In [ ]:
categories = os.listdir(train_directory)
print(str(len(categories)),'CATEGORIES are ', categories)

category_count = len(categories)

4 CATEGORIES are  ['audi a7', 'bmw serie 7', 'dodge charger', 'porsche 911']


In [ ]:
augmented_gen = ImageDataGenerator(
    rescale=1./255)

general_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = general_datagen.flow_from_directory(
    train_directory,
    target_size = (224, 224),
    batch_size = 16
)
valid_generator = general_datagen.flow_from_directory(
    val_directory,
    target_size = (224, 224),
    batch_size = 16
)
test_generator = general_datagen.flow_from_directory(
    test_directory,
    target_size = (224, 224),
    batch_size = 16
)

Found 4000 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.


In [ ]:
train_groups = len(train_generator)
valid_groups = len(valid_generator)
test_groups = len(test_generator)
print(f"Train groups: {train_groups}")
print(f"Validation groups: {valid_groups}")
print(f"Test groups: {test_groups}")

Train groups: 250
Validation groups: 63
Test groups: 63


In [ ]:
def conv_layer(inputs, filters, kernel_size=3, padding="valid"):
    x = layers.Conv2D(filters = filters, kernel_size = kernel_size, padding = padding, use_bias = False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    return x
# pooling layer i added dropout cause it help my model to reduce the overfitting
def pooling_layer(inputs, pool_size = 2, dropout_rate=0.5):
    x = layers.MaxPooling2D(pool_size = pool_size)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    return x

# this dense layer i will not only use it for my base model i will use it in the pretrained model too
def dense_layer(inputs, out, dropout_rate = 0.5):
    x = layers.Dense(out)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout_rate)(x)

    return x

In [ ]:
keras.backend.clear_session()

inputs = keras.Input(shape = (224, 224, 3))

x = conv_layer(inputs, 64, padding = "same")  # 224x224
x = conv_layer(x, 64)                         # 222x222
x = pooling_layer(x)                          # 111x111

x = conv_layer(x, 64, padding = "same")       # 111x111
x = conv_layer(x, 64)                         # 109x109
x = pooling_layer(x)                          # 54x54

x = conv_layer(x, 64, padding = "same")       # 54x54
x = conv_layer(x, 64)                         # 52X52
x = pooling_layer(x)                          # 26x26

x = conv_layer(x, 64, padding = "same")       # 54x54
x = conv_layer(x, 64)                         # 52X52
x = pooling_layer(x)

x = conv_layer(x, 64, padding = "same")       # 26x26

x = layers.Flatten()(x)                       # 26*26*64

x = dense_layer(x, 128)

outputs = layers.Dense(category_count, activation = "softmax")(x)

base_model = keras.Model(inputs, outputs)

base_model.summary()

In [ ]:
base_model.compile(optimizer =keras.optimizers.Adam(learning_rate=0.001),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
#fit model
history = base_model.fit(
    train_generator,
    steps_per_epoch = train_groups,
    epochs = 20, # adding more epochs will increase the acc like 1% or 2%
    validation_data = valid_generator,
    validation_steps = valid_groups,
    verbose = 1,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 3, restore_best_weights = True),
               keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 2, verbose = 1),
    keras.callbacks.ModelCheckpoint(
            filepath = "/content/drive/MyDrive/Datasets/carro pronto/intial_model.h5",
            save_best_only = True,
            monitor = "val_loss")
    ])

Epoch 1/20
250/250 [==============================] - ETA: 0s - loss: 1.4828 - accuracy: 0.3450 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 3129s 12s/step - loss: 1.4828 - accuracy: 0.3450 - val_loss: 1.4290 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 2/20
207/250 [=======================>......] - ETA: 7:46 - loss: 1.2023 - accuracy: 0.4973

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/Datasets/carro pronto/intial_model.h5")


test_results = model.evaluate(test_generator)


loss, accuracy = test_results

print(f'Perda (Loss): {loss}')
print(f'Acurácia: {accuracy}')

63/63 [==============================] - 6s 86ms/step - loss: 0.3644 - accuracy: 0.8690
Perda (Loss): 0.3644414246082306
Acurácia: 0.8690000176429749


In [ ]:
predictions = model.predict(test_generator)


predicted_classes = np.argmax(predictions, axis=1)


true_classes = test_generator.classes


from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Precisão: {accuracy}')

63/63 [==============================] - 6s 88ms/step
Precisão: 0.241


In [ ]:
from sklearn.metrics import f1_score
import numpy as np


predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)


true_classes = test_generator.classes


f1 = f1_score(true_classes, predicted_classes, average='weighted')

print(f'F1-Score: {f1}')

63/63 [==============================] - 5s 72ms/step
F1-Score: 0.25610437662882385


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)


true_classes = test_generator.classes


confusion = confusion_matrix(true_classes, predicted_classes)

print('Matriz de Confusão:')
print(confusion)

63/63 [==============================] - 8s 135ms/step
Matriz de Confusão:
[[73 56 63 58]
 [71 63 63 53]
 [68 51 81 50]
 [59 59 75 57]]
